In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


### View The Data
I will start by seeing what we got from our data collection. How many rows have location data and what does this data look like.

In [2]:
mydata = "tweets1018913.csv"
df = pd.read_csv(mydata)
df.head(5)

,Unnamed: 0,t_id,s_name,t_text,u_location,image_url,t_date
0,0.0,1052370940128968705,brandycooklyn,THINGS WRITERS LOVE:\n\n- Panel discussions.\n...,"Melbourne, Australia",http://pbs.twimg.com/profile_images/1042972658...,2018-10-17 01:29:18
1,1.0,1052370675632091136,Louroulai,RT @nelsonlflores: Tips for avoiding imposter ...,NaN,http://pbs.twimg.com/profile_images/2379147043...,2018-10-17 01:28:15
2,2.0,1052370144180211719,_WayneJ,RT @nelsonlflores: Tips for avoiding imposter ...,GA,http://pbs.twimg.com/profile_images/1050602456...,2018-10-17 01:26:09
3,3.0,1052369947236724736,Imposter_Cat,RT @isaidimhip: Anyway........anybody else fee...,impostersyndrome.life,http://pbs.twimg.com/profile_images/1047849327...,2018-10-17 01:25:22
4,4.0,1052369944506068992,isaidimhip,Anyway........anybody else feeling that impost...,NaN,http://pbs.twimg.com/profile_images/1021102005...,2018-10-17 01:25:21


In [3]:
df.dtypes

Unnamed: 0    float64
t_id           object
s_name         object
t_text         object
u_location     object
image_url      object
t_date         object
dtype: object

In [4]:
df.shape

(26819, 7)

In [5]:
df.apply(lambda x: sum(x.isnull()),axis=0)

Unnamed: 0     286
t_id             0
s_name           0
t_text           0
u_location    4646
image_url        0
t_date           0
dtype: int64

In [6]:

df['u_location'].value_counts(ascending=False).head(20)

impostersyndrome.life    4552
London, England           387
u_location                286
London                    256
Los Angeles, CA           220
United States             178
Chicago, IL               166
Washington, DC            151
Boston, MA                141
New York, NY              140
United Kingdom            138
Houston, TX               124
California, USA           117
UK                        104
Seattle, WA               100
Atlanta, GA                99
Brooklyn, NY               88
San Francisco, CA          85
Philadelphia, PA           83
Portland, OR               81
Name: u_location, dtype: int64

In [7]:
df['u_location'].value_counts(ascending=False).tail(20)

Lahti, Suomi                      1
DC/Boston/New Haven               1
Los                               1
A noisy MDF                       1
Soul searching✨                   1
Beantown                          1
dnd                               1
ptx/frisco/Hunstvegas             1
$C                                1
NorCal                            1
Staten Island, NY.                1
Sunderland UK                     1
Old Dominion University ‘21       1
661-415                           1
Sheffield                         1
IG: julia_victoriaa               1
Retweets are not Endorsements     1
Rockville, MD                     1
Disbelief, USA                    1
Bethlehem, PA                     1
Name: u_location, dtype: int64

## What are we looking for?
Ok, some places are real and some are not. Looking at the data I am going to stick with locations in the USA and locations that have been picked via the drop-down menu on twitter. Here is what they look like:
#### City, State:
Maplewood, NJ
#### State, County:
New Jersey, USA
#### Country:
United States

1. Entries will have a single comma or be "United States"
2. Full city name with a state abbriviation
3. Full state name with a country abbriviation
4. Full country name

#### What we want, is to convert it to one of the following:

Maplewood, New Jersey, USA

New Jersey, USA

USA

## Cleaning
1. Remove empty location rows
2. Convert NJ to New Jersey, USA
3. Convert United States to USA
4. Keep only fileds that have USA in them
5. Removing retweets

In [8]:
# mark if a retweet and then drop
df['get_rt'] = df['t_text'].str.startswith('RT')
df.drop(df[df['get_rt'] == True].index, inplace=True)
df.shape

(7795, 8)

That was a lot of retweets the number of original tweets is 7795. Next we drop all empty locations and take a look at our data again.

In [9]:
df.dropna(subset=['u_location'], how='all', inplace = True)

In [10]:
df.apply(lambda x: sum(x.isnull()),axis=0)

Unnamed: 0    286
t_id            0
s_name          0
t_text          0
u_location      0
image_url       0
t_date          0
get_rt          0
dtype: int64

In [11]:
df.shape

(6561, 8)

We started with a little over 26k tweets and are down to around 7k. Now we will convert NJ to New Jersey, USA and United States to USA. I do this using a dictionary for all of the states.

In [12]:
us_state_abbrev = {
    'AL': 'Alabama, USA',
    'AK': 'Alaska, USA',
    'AZ': 'Arizona, USA',
    'AR': 'Arkansas, USA',
    'CA': 'California, USA',
    'CO': 'Colorado, USA',
    'CT': 'Connecticut, USA',
    'DE': 'Delaware, USA',
    'FL': 'Florida, USA',
    'GA': 'Georgia, USA',
    'HI': 'Hawaii, USA',
    'ID': 'Idaho, USA',
    'IL': 'Illinois, USA',
    'IN': 'Indiana, USA',
    'IA': 'Iowa, USA',
    'KS': 'Kansas, USA',
    'KY': 'Kentucky, USA',
    'LA': 'Louisiana, USA',
    'ME': 'Maine, USA',
    'MD': 'Maryland, USA',
    'MA': 'Massachusetts, USA',
    'MI': 'Michigan, USA',
    'MN': 'Minnesota, USA',
    'MS': 'Mississippi, USA',
    'MO': 'Missouri, USA',
    'MT': 'Montana, USA',
    'NE': 'Nebraska, USA',
    'NV': 'Nevada, USA',
    'NH': 'New Hampshire, USA',
    'NJ': 'New Jersey, USA',
    'NM': 'New Mexico, USA',
    'NY': 'New York, USA',
    'NC': 'North Carolina, USA',
    'ND': 'North Dakota, USA',
    'OH': 'Ohio, USA',
    'OK': 'Oklahoma, USA',
    'OR': 'Oregon, USA',
    'PA': 'Pennsylvania, USA',
    'RI': 'Rhode Island, USA',
    'SC': 'South Carolina, USA',
    'SD': 'South Dakota, USA',
    'TN': 'Tennessee, USA',
    'TX': 'Texas, USA',
    'UT': 'Utah, USA',
    'VT': 'Vermont, USA',
    'VA': 'Virginia, USA',
    'WA': 'Washington, USA',
    'WV': 'West Virginia, USA',
    'WI': 'Wisconsin, USA',
    'WY': 'Wyoming, USA',
    'United States': 'USA'
}

In [13]:
# Convert 2 letter abbrv. to State name, USA
df['u_location'] = df['u_location'].replace(us_state_abbrev, regex=True)

In [15]:
# mark all with the count of USA and grab those with only one.
usa_counts = df['u_location'].str.count('USA')
# usa_counts
df['with_usa'] = np.where((usa_counts == 1),df.u_location, "")

In [16]:
df['with_usa'].value_counts(ascending=False).tail(10)

Bayside, New York, USA                        1
NOLouisiana, USA✈️DC                          1
Knoxville, Tennessee, USA                     1
Oregon, USA                                   1
My Mind, Florida, USA                         1
Columbus, Indiana, USA                        1
Kansas, USA                                   1
Windsor Hills, California, USA                1
Beaufort South Carolina, USA | Oxford U.K.    1
Davenport, Florida, USA                       1
Name: with_usa, dtype: int64

In [17]:
df['with_usa'].value_counts(ascending=False).head(20)

                                  4346
Los Angeles, California, USA       111
New York, New York, USA             81
Chicago, Illinois, USA              70
USA                                 66
California, USA                     63
Oak Park, Illinois, USA             61
Seattle, Washington, USA            61
San Francisco, California, USA      51
Illinois, USA                       46
Brooklyn, New York, USA             42
Louisville, Kentucky, USA           41
New York, USA                       38
Boston, Massachusetts, USA          34
Davis, California, USA              34
District of Columbia, USA           33
Austin, Texas, USA                  32
Portland, Oregon, USA               31
Oakland, California, USA            31
Houston, Texas, USA                 31
Name: with_usa, dtype: int64

#### Evaluate
This is much better but we are still getting some krust, I am going to see if I can clean this up a bit more by removing any entry that has a special char. or does not end in USA

In [18]:
# Must end with USA
end_usa = df['u_location'].str.endswith('USA')
df['with_usa'] = np.where((end_usa == True),df.with_usa, "")
#df['with_usa'].value_counts(ascending=False).tail(120)

In [19]:
# No junk in the location field
df['with_usa'] = df['with_usa'].str.replace('[^a-zA-Z\s,]', '')


In [20]:
# make sure all fields are populated
df.dropna(inplace = True)

In [25]:
df['usa_count'] = df['with_usa'].str.count('USA')
df.drop(df[df['usa_count'] == 0].index, inplace=True)


In [26]:
# we can only have up to two commas, drop rows with more
comma_counts = df['with_usa'].str.count(',')
df['with_comma'] = np.where((comma_counts <= 2),df.with_usa, "")
df.shape


(2134, 11)

This takes us down to 2134 tweets that have pretty clean location information.

#### Split into three different rows
I split the location data into three rows with a split in reverse order. This orders the data with a left justify. I then strip each row of extra spaces and drop the column we do not need. Finally I write out the new csv file.

In [27]:
# split into three fields
df[['Country', 'State', 'City']] = df['with_comma'].apply(lambda x:pd.Series(x.split(",")[::-1]))

In [28]:
# strip extra spaces
df['Country'] = df['Country'].str.strip()
df['State'] = df['State'].str.strip()
df['City'] = df['City'].str.strip()


In [29]:
# drop unused columns
df.drop(['with_usa', 'usa_count', 'with_comma', 'get_rt'], axis=1)


,Unnamed: 0,t_id,s_name,t_text,u_location,image_url,t_date,Country,State,City
7,7.0,1052369469580820481,hayleydenb,True story. I had a dream last night that I wa...,"San Diego, California, USA",http://pbs.twimg.com/profile_images/1047347755...,2018-10-17 01:23:28,USA,California,San Diego
16,16.0,1052367305726865408,molls,@katieferg @kimberlylhudson If you're being st...,"Los Angeles, California, USA",http://pbs.twimg.com/profile_images/1046674961...,2018-10-17 01:14:52,USA,California,Los Angeles
79,79.0,1052357068118740992,Hentai4Dinner,Does anybody else know what imposter syndrome ...,"Springfield, Missouri, USA",http://pbs.twimg.com/profile_images/1033836023...,2018-10-17 00:34:11,USA,Missouri,Springfield
81,81.0,1052356563690840065,emilymias,I had the privilege today to talk to @MassArt...,"Boston, Massachusetts, USA",http://pbs.twimg.com/profile_images/9375178224...,2018-10-17 00:32:11,USA,Massachusetts,Boston
91,91.0,1052354795632250881,DopeBlckQueen,this transition to graduate school has been on...,"Raleigh, North Carolina, USA",http://pbs.twimg.com/profile_images/1040736565...,2018-10-17 00:25:09,USA,North Carolina,Raleigh
102,1.0,1052352901648125957,kanavjain,and @r2rothenberg talks to us about how he got...,"New York, New York, USA",http://pbs.twimg.com/profile_images/1038427834...,2018-10-17 00:17:38,USA,New York,New York
108,7.0,1052352462844133377,gmksp,Imposter syndrome https://t.co/evAj9wZC0z,"Tempe, Arizona, USA",http://pbs.twimg.com/profile_images/1050619495...,2018-10-17 00:15:53,USA,Arizona,Tempe
118,17.0,1052349532871188480,jimtoepel,@Hellchick Spoiler: It's imposter syndrome all...,"Redmond, Washington, USA",http://pbs.twimg.com/profile_images/9004442737...,2018-10-17 00:04:15,USA,Washington,Redmond
151,50.0,1052345893200904197,_markryann,#NationalBossDay \nI have the worst imposter s...,"Boncord, California, USA",http://pbs.twimg.com/profile_images/1040674494...,2018-10-16 23:49:47,USA,California,Boncord
153,52.0,1052345782676975616,IamAleshiaP,@CRWNmag @Travepreneur I dealt with Imposter S...,"Saint Louis, Missouri, USA",http://pbs.twimg.com/profile_images/9349134392...,2018-10-16 23:49:20,USA,Missouri,Saint Louis


In [30]:
df.shape

(2134, 14)

In [31]:
# write out my file of clean tweets
df.to_csv('tweets_clean_1018.csv')